In [1]:
import requests
from datetime import datetime
from dateutil import tz
import pandas as pd
import re

cookies = {
    'YOUR_COOKIES': ''
}

headers = {
    'YOUR_HEADERS': ''
}

params = {
    'sesskey': 'SESS_KEY',
    'info': 'core_calendar_get_calendar_monthly_view',
}

json_data = [
    {
        'index': 0,
        'methodname': 'core_calendar_get_calendar_monthly_view',
        'args': {
            'year': '2024',
            'month': '9',
            'courseid': 1,
            'day': 1,
            'view': 'month',
        },
    },
]

response = requests.post('https://edu.hse.ru/lib/ajax/service.php',
                         params=params, cookies=cookies, headers=headers, json=json_data)


def get_date(input_string):
    # Regular expression to find the time
    pattern = r'(\d{2}:\d{2})'

    # Search for the pattern in the input string
    match = re.search(pattern, input_string)

    if match:
        return match.group(1)  # Extract the matched time
    else:
        return "No match found"


def date_fromatter(date_str, lang='ru'):
    if lang == 'ru': 
        res = [val.capitalize()
            for val in date_str.replace('События - ', '').split()]
        return f"{res[0]}, {' '.join(res[1:])}"
    elif lang == 'en': 
        return ' '.join([val.strip().capitalize() for val in date_str.replace('events', '').strip().split()])


def to_flat(x): return [i for sub in x for i in sub]


### Json parsing
data = response.json()[0].get('data', {})
period = data.get('periodname', '')
events = [[day for day in week.get('days', []) if day.get('hasevents')] for week in data.get('weeks', [])]
events_flat = to_flat(events)
events_small = [[[[ event.get('viewdaylink', ''),
                    str(datetime.fromtimestamp(event.get('timestamp'), tz=tz.gettz("GMT+3"))),
                    event.get('viewdaylinktitle', ''),
                    e.get('activityname', ''),
                    e.get('viewurl'),
                    e.get('url',  ''),
                    e.get('description', ''),
                    e.get('formattedtime'),
                    e.get('course', {}).get('fullnamedisplay'),
                    e.get('course', {}).get('viewurl', '')] for e in event.get('events', [{}])]
                ] for event in events_flat]
result = to_flat(to_flat(events_small))

### Pandas output
df = pd.DataFrame(result)
df.columns = ['calendar_link', 'date', 'date_str', 'name', 'calendar',
              'hw_link', 'description', 'due_time', 'subj', 'subj_link']
output = f'Время последнего обновления: {str(datetime.now())[:19]} \n'
df = df.sort_values('date').reset_index(drop=True)
for dt in df.date_str.unique():
    output += '\n' + date_fromatter(dt) + '\n'
    cur_df = df[df.date_str == dt]
    for idx in range(len(cur_df)):
        output += f'{cur_df.iloc[idx].subj} {cur_df.iloc[idx].subj_link}  \n'
        output += f"{cur_df.iloc[idx]['name']} {cur_df.iloc[idx].hw_link} \n"
        output += f'До {get_date(cur_df.iloc[idx].due_time)} \n'
        
### Result
print(output)

Время последнего обновления: 2024-09-11 12:05:57 

Суббота, 14 Сентября
Python для инженерии данных (2024/2025 модули: 1,2) https://edu.hse.ru/course/view.php?id=211078  
Домашнее задание 1 https://edu.hse.ru/mod/assign/view.php?id=1280150 
До 23:59 

Понедельник, 16 Сентября
Информационные системы (2024/2025 модули: 1,2) https://edu.hse.ru/course/view.php?id=211074  
Домашнее задание по теме 1 «Введение в информационные системы» https://edu.hse.ru/mod/assign/view.php?id=1296563 
До 16:00 
Семинар наставника (2024/2025 модули: 1,2,3,4) https://edu.hse.ru/course/view.php?id=211075  
Паспорт компетенций https://edu.hse.ru/mod/assign/view.php?id=1290223 
До 23:59 

Понедельник, 23 Сентября
Информационные системы (2024/2025 модули: 1,2) https://edu.hse.ru/course/view.php?id=211074  
Домашнее задание по теме 2 «Методологии и инструменты создания информационных систем» https://edu.hse.ru/mod/assign/view.php?id=1296574 
До 16:00 

Суббота, 28 Сентября
Python для инженерии данных (2024/2025 мо

In [ ]:
# tg @nikitaworonov для доработок
# TODO Подумать можно ли это все дело как - то удобно кидать в ТГ
# TODO Может развернуть и захостить все это в кроне
# TODO Может по файликам лучше в проект отдельные это дело раскидать